In [ ]:
# 데이터 전처리 파일

In [107]:
import numpy as np
import pandas as pd

In [ ]:
!pip install transformers

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv")

In [3]:
df.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [9]:
# 데이터 프레임 정보 요약
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11823 entries, 0 to 11822
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Q       11823 non-null  object
 1   A       11823 non-null  object
 2   label   11823 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 277.2+ KB


In [11]:
# 결측값 정보를 출력합니다.
df.isnull().sum()

Q        0
A        0
label    0
dtype: int64

In [23]:
df_need = df[df.label != 0] # label이 0인 값 제거

In [24]:
df_need

,Q,A,label
5290,1000일 만난 여자친구와 이별,더 오래 만날 사람 만날 거예요.,1
5291,10년 연애. 헤어졌습니다.,더 공허함이 크시겠네요.,1
5292,10년 연애사 되돌아보니 다 부질없네,더 좋은 사람 만나실 거예요.,1
5293,10년 연예의끝,더 마음이 허하겠어요.,1
5294,10년만나다 헤어지네,충분히 슬퍼하고 충분히 아파하다가 이겨내세요.,1
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


In [26]:
df_shuffled = df_need.sample(frac=1).reset_index(drop=True)

In [33]:
train_data = list(df_shuffled.Q)[:4000]
test_data = list(df_shuffled.Q)[4000:5000]

In [30]:
train_label = list(df_shuffled.label)[:4000]
test_label = list(df_shuffled.label)[4000:5000]

In [35]:
train_data[:5]

['좋아하는 사람 앞에서 실례를 했어.', '프사를 왜 보았을까', '답답해서', '짝사랑 해봤어?', '지난주 이별했네']

In [36]:
train_label[:5]

[2, 1, 1, 2, 1]

In [39]:
from transformers import BertTokenizer, BertModel

/home/elicer/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
tokenizer_config.json: 100%|██████████| 29.0/29.0 [00:00<00:00, 183kB/s]
vocab.txt: 100%|██████████| 996k/996k [00:00<00:00, 1.26MB/s]
tokenizer.json: 100%|██████████| 1.96M/1.96M [00:00<00:00, 9.22MB/s]
config.json: 100%|██████████| 625/625 [00:00<00:00, 1.63MB/s]


In [48]:
# 토크나이저 불러오기
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)

In [72]:
# 최대값 구하기(최대 시퀀스 길이 설정할 때 참고용)
temp = 0
for _, sentence in enumerate(input_ids):
    temp = max(temp, len(sentence))

print(temp)

# sequence 최대 길이가 39였기 때문에 여유를 두고 60으로 설정했습니다
MAX_LEN = 60 #최대 시퀀스 길이 설정

39


In [75]:
# pad_sequences? Sequences로 이루어진 리스트를 Numpy 2d 배열로 바꿔주는 함수
from keras.utils import pad_sequences

In [88]:
def tokenize_list(data):
    sentences = ["[CLS] " + str(s) + " [SEP]" for s in data]
    tokenized_texts = [tokenizer.tokenize(s) for s in sentences]

    print("토크나이징 전: ", sentences[0])
    print("토크나이징 후: ", tokenized_texts[0])

    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    return input_ids

def transform_list(input_ids):
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
    return input_ids

In [92]:
test_input_ids = tokenize_list(test_data)
test_input_ids = transform_list(test_input_ids)
print(len(input_ids), len(test_input_ids))

토크나이징 전:  [CLS] 장거리연애였는데 [SEP]
토크나이징 후:  ['[CLS]', '장', '##거', '##리', '##연', '##애', '##였', '##는데', '[SEP]']
4000 1000


In [98]:
input_ids[0]

array([  101,  9685, 16985, 12178,  9405, 61250,  9531, 11489,  9489,
       58762, 11513,  9965, 12965,   119,   102,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0])

In [87]:
# attention mask : padding된 값(즉, 0)을 구별할 수 있도록 생성
attention_masks = []
print(len(input_ids))
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

4000


In [99]:
seq_mask[50]

0.0

In [109]:
np.save('./train_x', input_ids)
np.save('./test_x', test_input_ids)

In [115]:
np.save('./train_y', train_label)
np.save('./test_y', test_label)

In [110]:
train_x = np.load('train_x.npy') # 파일 불러와서 확인

In [103]:
# "sangrimlee/bert-base-multilingual-cased-nsmc"
# model = TFBertModel.from_pretrained("bert-base-multilingual-cased")